In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import src.config as config

In [3]:
import hopsworks

# connect to project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)
project

2025-02-04 15:51:18,333 INFO: Initializing external client
2025-02-04 15:51:18,334 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-04 15:51:19,994 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211558


Project('nyc_taxi_demand_forecast', 'sushmithahs047@gmail.com', 'Default project')

In [4]:
# connect to feature store
feature_store = project.get_feature_store()
feature_store

In [5]:
# connect to feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)
feature_group

In [6]:
# create a feature view 

try:
    # create a feature view if ti doesnt exist
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feature view already exists. Skip creation")

Feature view already exists. Skip creation


In [7]:
# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)
feature_view

In [8]:
# fetch data from feature store using feature view
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides'
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (50.61s) 


In [9]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2023-06-20 04:00:00+00:00,6,100
1,2022-04-29 07:00:00+00:00,54,249
2,2024-04-10 13:00:00+00:00,0,128
3,2023-04-18 11:00:00+00:00,0,136
4,2022-05-28 03:00:00+00:00,0,62
...,...,...,...
6895651,2024-03-28 22:00:00+00:00,0,124
6895652,2023-03-06 16:00:00+00:00,101,107
6895653,2024-10-21 01:00:00+00:00,0,135
6895654,2024-09-26 19:00:00+00:00,0,106


In [10]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
728110,2022-01-01 00:00:00+00:00,0,1
483537,2022-01-01 01:00:00+00:00,0,1
1557562,2022-01-01 02:00:00+00:00,0,1
2320817,2022-01-01 03:00:00+00:00,0,1
5380296,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
3887279,2025-02-03 13:00:00+00:00,2,265
4719581,2025-02-03 14:00:00+00:00,2,265
4350747,2025-02-03 15:00:00+00:00,2,265
4111405,2025-02-03 16:00:00+00:00,1,265


### Converting timeseries data into feature and target

In [11]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 263/263 [01:58<00:00,  2.23it/s]


features_and_target.shape=(292311, 675)


In [12]:
import pandas as pd
features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour']).dt.tz_convert('UTC')
print(f'{features_and_target.pickup_hour.dtype=}')

features_and_target.pickup_hour.dtype=datetime64[ns, UTC]


In [13]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up untill 3 months ago
# testing data -> last 3 months

cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*3)).tz_localize('UTC')


print(f'{cutoff_date=}')

cutoff_date=Timestamp('2024-11-12 00:00:00+0000', tz='UTC')


In [14]:
X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(279569, 674)
y_train.shape=(279569,)
X_test.shape=(12742, 674)
y_test.shape=(12742,)


### Training LightGBM model

In [15]:
from src.model import get_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np

from src.model import get_pipeline

import optuna
def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average validation error based on a TimeSeriesSplit
    """

    # pick hyper-parameters
    hyperparams = {
        "metric": "mae",
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    
    # return the mean score
    return np.array(scores).mean()

In [16]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2025-02-04 15:55:08,965] A new study created in memory with name: no-name-4e92a694-8c09-4555-b621-17767b606e21
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [17]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 55, 'feature_fraction': 0.8757911583438946, 'bagging_fraction': 0.8513497132265813, 'min_child_samples': 82}


In [18]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function avg_rides_last_4_weeks at 0x384b809a0>)),
                ('temporalfeatureengineering', TemporalFeatureEngineering()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8513497132265813,
                               feature_fraction=0.8757911583438946,
                               min_child_samples=82, num_leaves=55))])

In [19]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.5781


In [20]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/sushmitha047/Documents/RealWorldML-Pau/taxi_demand_predictor/models/model.pkl']

### Hopsworks Model schema - format for the input data and output data
here we are providing example of input schema and output schema using X_train and y_train

In [21]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

### Hopsworks Model Registry

In [22]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_forecaster_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor wiht a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR / 'model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/556247 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3409 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/58132 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1211558/models/taxi_demand_forecaster_next_hour/2


Model(name: 'taxi_demand_forecaster_next_hour', version: 2)